In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import rasters as rt
from PTJPLSM import PTJPLSM, process_PTJPLSM_table
from PTJPL import load_Topt, load_fAPARmax
from soil_capacity_wilting import load_field_capacity
from soil_capacity_wilting import load_wilting_point
from gedi_canopy_height import load_canopy_height

In [2]:
tower_data_filename = "combined_eco_flux_EC_filtered.csv"
tower_locations_filename = "metadata_ebc_filt.csv"

In [3]:
generated_input_table_filename = "ECOv002-cal-val-PT-JPL-SM-inputs.csv"

In [4]:
tower_locations_df = pd.read_csv(tower_locations_filename)
tower_locations_df

,Site ID,Name,Lat,Long,Elev,Clim,Veg,MAT,MAP,StartDate,EndDate,LE_count,closure_ratio
0,US-NC3,NC_Clearcut#3,35.7990,-76.6560,5.0,Cfa,ENF,16.6,1320.00,10/1/18 05:00,1/1/22 05:00,9576,1.02
1,PE-QFR,Quistococha Forest Reserve,-3.8344,-73.3190,104.0,Af,WET,NaN,NaN,10/1/18 05:00,1/1/20 05:00,6859,0.98
2,US-Mi3,LTAR UCB (Upper Chesapeake Bay) Miscanthus 3,41.8222,-80.6370,270.0,Dfb,CVM,10.5,1012.70,10/1/18 05:00,12/28/19 04:00,12170,0.92
3,US-NC4,NC_AlligatorRiver,35.7879,-75.9038,1.0,Cfa,WET,16.6,1311.00,10/1/18 05:00,1/1/22 05:00,20890,0.90
4,CA-DB2,Delta Burns Bog 2,49.1190,-122.9951,4.0,NaN,WET,NaN,NaN,1/1/19 08:30,1/1/21 08:00,11884,0.89
...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,US-xSL,"NEON North Sterling, CO (STER)",40.4619,-103.0293,1364.0,Bsk,CRO,9.7,432.67,10/1/18 07:00,12/1/22 00:00,23176,0.60
117,US-xWD,NEON Woodworth (WOOD),47.1282,-99.2414,579.0,Dfb,GRA,4.9,493.76,10/1/18 06:00,12/1/22 00:00,20494,0.60
118,US-CS4,Central Sands Irrigated Agricultural Field,44.1597,-89.5475,328.0,Dfa,CRO,7.0,830.00,1/1/20 06:30,4/27/21 06:00,6359,0.60
119,US-xAE,NEON Klemme Range Research Station (OAES),35.4106,-99.0588,516.0,Cfa,GRA,15.5,778.85,10/1/18 06:00,12/1/22 00:00,29615,0.60


In [5]:
tower_IDs = list(tower_locations_df["Site ID"])
tower_IDs

['US-NC3',
 'PE-QFR',
 'US-Mi3',
 'US-NC4',
 'CA-DB2',
 'US-Sne',
 'US-Mi1',
 'US-PFe',
 'US-NR1',
 'US-Vcp',
 'US-xAB',
 'US-HBK',
 'US-EDN',
 'US-PFh',
 'US-Me6',
 'US-NC2',
 'US-Whs',
 'US-SRM',
 'US-CS1',
 'US-PFs',
 'US-PFg',
 'US-Ha1',
 'US-Ne2',
 'US-PFm',
 'US-PFr',
 'US-PFL',
 'US-PFj',
 'US-CC2',
 'US-NR3',
 'US-PFt',
 'CA-Cbo',
 'PR-xLA',
 'US-Ne3',
 'US-PHM',
 'US-Tw1',
 'US-Hn3',
 'US-PFk',
 'US-CS2',
 'US-ONA',
 'US-CC1',
 'US-Rls',
 'US-UMd',
 'US-PFq',
 'US-PFn',
 'US-HB3',
 'US-DFC',
 'US-Wkg',
 'US-CF1',
 'US-Ho2',
 'US-Snf',
 'US-Me2',
 'US-CF3',
 'US-Ne1',
 'US-Ha2',
 'CA-Ca3',
 'US-SP1',
 'US-UMB',
 'US-Bi1',
 'US-PFi',
 'US-CF2',
 'US-KFS',
 'US-CMW',
 'US-Ro4',
 'US-PAS',
 'US-xYE',
 'US-Los',
 'US-PFd',
 'US-KM4',
 'US-Syv',
 'US-WCr',
 'US-Rws',
 'US-xJE',
 'US-SRG',
 'US-xNW',
 'US-Rms',
 'US-NR4',
 'US-xDS',
 'US-xJR',
 'US-Bar',
 'US-ALQ',
 'US-MMS',
 'US-UC1',
 'US-PFb',
 'US-xDL',
 'US-xPU',
 'US-xBL',
 'US-xST',
 'US-xSB',
 'US-xRN',
 'US-UC2',
 'US-xTR',

In [6]:
tower_names = list(tower_locations_df.Name)
tower_names

['NC_Clearcut#3',
 'Quistococha Forest Reserve',
 'LTAR UCB (Upper Chesapeake Bay) Miscanthus 3',
 'NC_AlligatorRiver',
 'Delta Burns Bog 2',
 'Sherman Island Restored Wetland',
 'LTAR UCB (Upper Chesapeake Bay) Miscanthus 1',
 'NW4 Lake-1 CHEESEHEAD 2019',
 'Niwot Ridge Forest (LTER NWT1)',
 'Valles Caldera Ponderosa Pine',
 'NEON Abby Road (ABBY)',
 'Hubbard Brook Experimental Forest',
 'Eden Landing Ecological Reserve',
 'NE2 Pine-3 CHEESEHEAD 2019',
 'Metolius Young Pine Burn',
 'NC_Loblolly Plantation',
 'Walnut Gulch Lucky Hills Shrub',
 'Santa Rita Mesquite',
 'Central Sands Irrigated Agricultural Field',
 'SE5 Aspen-5 CHEESEHEAD 2019',
 'NE1 Pine-2 CHEESEHEAD 2019',
 'Harvard Forest EMS Tower (HFR1)',
 'Mead - irrigated maize-soybean rotation site',
 'SW3 Hardwood-2 CHEESEHEAD 2019',
 'SE4 Tussock-2 CHEESEHEAD 2019',
 'SW2 Aspen-3 CHEESEHEAD 2019',
 'NE4 Maple-1 CHEESEHEAD 2019',
 'Coloma Corn 2',
 'Niwot Ridge Alpine (T-Van West)',
 'SE6 Pine-4 CHEESEHEAD 2019',
 'Ontario - Mi

In [7]:
tower_data_df = pd.read_csv(tower_data_filename)
tower_data_df

,Unnamed: 0,ID,vegetation,climate,STICinst,BESSinst,MOD16inst,PTJPLSMinst,ETinst,ETinstUncertainty,...,ESIrn_STIC,ESIrn_PTJPLSM,ESIrn_MOD16,ESIrn_BESS,ESIrn_Unc_ECO,ESIrn_LEcorr50,JET,eco_time_utc,Site Name,Date-Time
0,0,US-NC3,ENF,Cfa,270.345200,78.53355,392.851840,307.021970,487.383423,118.916280,...,0.686404,0.779526,0.997448,0.199396,0.301927,0.737734,288.683585,2019-10-02 19:09:40,US-NC3,2019-10-02 19:09:40
1,1,US-Mi3,CVM,Dfb,232.141600,229.20093,640.118470,375.089300,106.825577,167.919460,...,0.360762,0.582912,0.994784,0.356192,0.260957,0.413558,303.615450,2019-06-23 18:17:17,US-Mi3,2019-06-23 18:17:17
2,2,US-Mi3,CVM,Dfb,356.355740,335.23154,625.661700,284.686250,NaN,132.936340,...,0.566752,0.452768,0.995059,0.533156,0.211423,0.558382,345.793640,2019-06-27 16:35:42,US-Mi3,2019-06-27 16:35:42
3,3,US-Mi3,CVM,Dfb,332.938400,326.68680,624.254330,251.414490,178.827545,141.132420,...,0.531036,0.401006,0.995685,0.521065,0.225106,0.501352,329.812600,2019-06-30 15:44:10,US-Mi3,2019-06-30 15:44:10
4,4,US-Mi3,CVM,Dfb,286.854030,237.21654,511.082180,228.520170,154.791626,114.809410,...,0.559024,0.445342,0.996002,0.462290,0.223742,0.375202,262.035285,2019-07-01 14:53:48,US-Mi3,2019-07-01 14:53:48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1060,1060,US-xAE,GRA,Cfa,70.923310,172.37459,81.645230,15.282976,NaN,56.385185,...,0.434453,0.093619,0.500132,1.055910,0.345397,0.054837,76.284270,2021-12-11 16:01:12,US-xAE,2021-12-11 16:01:12
1061,1061,US-xAE,GRA,Cfa,116.543190,121.81641,65.469320,22.186659,NaN,40.509410,...,0.701256,0.133500,0.393938,0.732986,0.243751,0.264294,91.006255,2022-03-25 22:45:31,US-xAE,2022-03-25 22:45:31
1062,1062,US-xAE,GRA,Cfa,129.880100,0.00000,118.777240,55.343586,NaN,52.403820,...,0.550616,0.234624,0.503546,0.000000,0.222162,0.355559,87.060413,2022-04-12 22:53:09,US-xAE,2022-04-12 22:53:09
1063,1063,US-xAE,GRA,Cfa,2.707851,140.38632,126.490524,40.434025,NaN,57.769722,...,0.010924,0.163120,0.510292,0.566350,0.233056,0.269973,83.462274,2022-04-14 14:45:37,US-xAE,2022-04-14 14:45:37


In [8]:
tower_data_df.columns

Index(['Unnamed: 0', 'ID', 'vegetation', 'climate', 'STICinst', 'BESSinst',
       'MOD16inst', 'PTJPLSMinst', 'ETinst', 'ETinstUncertainty', 'PET', 'Rn',
       'ESI', 'RH', 'Ta', 'LST', 'SM', 'NDVI', 'NDVI-UQ', 'albedo',
       'albedo-UQ', 'LST_err', 'view_zenith', 'Rg', 'EmisWB', 'time_utc',
       'solar_time', 'solar_hour', 'local_time', 'LE', 'LE_filt', 'LEcorr25',
       'LEcorr50', 'LEcorr75', 'LEcorr_ann', 'H_filt', 'Hcorr25', 'Hcorr50',
       'Hcorr75', 'Hcorr_ann', 'NETRAD_filt', 'G_filt', 'SM_surf', 'SM_rz',
       'AirTempC', 'SW_IN', 'RH_percentage', 'ESIrn_STIC', 'ESIrn_PTJPLSM',
       'ESIrn_MOD16', 'ESIrn_BESS', 'ESIrn_Unc_ECO', 'ESIrn_LEcorr50', 'JET',
       'eco_time_utc', 'Site Name', 'Date-Time'],
      dtype='object')

In [9]:
# Create MultiPoint geometry for tower locations
tower_points = rt.MultiPoint(
    x=tower_locations_df['Long'].values,
    y=tower_locations_df['Lat'].values
)

tower_points

MULTIPOINT ((-76.656 35.799), (-73.319 -3.8344), (-80.637 41.8222), (-75.9038 35.7879), (-122.9951 49.119), (-121.7547 38.0369), (-80.6313 41.7727), (-90.3004 45.9793), (-105.5464 40.0329), (-106.5974 35.8624), (-122.3303 45.7624), (-71.7181 43.9397), (-122.114 37.6156), (-90.2406 45.9557), (-121.6078 44.3233), (-76.6685 35.803), (-110.0522 31.7438), (-110.8661 31.8214), (-89.5379 44.1031), (-90.2382 45.9381), (-90.2723 45.9735), (-72.1715 42.5378), (-96.4701 41.1649), (-90.3099 45.9207), (-90.2475 45.9245), (-90.3177 45.9409), (-90.227 45.9619), (-89.6196 44.1039), (-105.5864 40.052), (-90.2288 45.9197), (-79.9333 44.3167), (-67.0769 18.0212), (-96.4397 41.1797), (-70.8301 42.7423), (-121.6469 38.1074), (-119.4614 46.6878), (-90.3425 45.9149), (-89.5002 44.1467), (-81.9509 27.3836), (-89.6787 44.0732), (-116.7356 43.1439), (-84.6975 45.5625), (-90.2475 45.9271), (-90.2823 45.9392), (-79.2322 33.3482), (-89.7117 43.3448), (-109.9419 31.7365), (-117.0821 46.7815), (-68.747 45.2091), (-1

In [10]:
Topt = load_Topt(geometry=tower_points)
Topt

array([10.09, 25.06,  1.39,  9.74,  3.43, 11.86,  2.06,  0.  ,  0.  ,
        0.16,  6.69,  0.  , 17.53,  0.  ,  0.  , 11.61, 17.87, 18.75,
        0.  ,  0.  ,  0.  ,  0.  ,  1.85,  0.  ,  0.  ,  0.  ,  0.  ,
        0.  ,  0.  ,  0.  ,  0.  , 23.22,  1.52,  3.01, 14.51, 12.82,
        0.  ,  0.  , 18.39,  0.  ,  0.  ,  0.  ,  0.  ,  0.  , 12.81,
        0.  , 15.57,  0.  ,  0.  , 10.83,  0.  ,  0.  ,  1.85,  0.  ,
        6.34, 15.93,  0.  , 13.84,  0.  ,  0.  ,  4.28, 12.27,  0.  ,
       18.39,  0.  ,  0.  ,  0.  ,  0.64,  0.  ,  0.  ,  0.  , 13.57,
       20.45,  0.  ,  0.  ,  0.  , 17.59, 11.84,  0.  ,  0.  ,  4.4 ,
        0.72,  0.  , 11.44,  0.  ,  4.79,  0.  , 15.67,  6.  ,  0.83,
        0.  ,  0.69, 14.51, 22.28, 12.14,  0.  ,  3.79,  0.  ,  5.8 ,
        0.  ,  0.  ,  0.  , 15.81,  3.43, 12.82,  0.  ,  4.6 ,  0.  ,
        1.44,  0.  ,  2.66,  2.58, 23.21,  8.48,  4.7 , 12.81,  0.66,
        0.  ,  0.  ,  7.99,  0.  ])

In [11]:
fAPARmax = load_fAPARmax(geometry=tower_points)
fAPARmax

array([0.4659, 0.5924, 0.4865, 0.5777, 0.4949, 0.3557, 0.4792, 0.4739,
       0.4462, 0.4323, 0.604 , 0.5306, 0.2956, 0.4958, 0.4851, 0.5663,
       0.2522, 0.2773, 0.4053, 0.4951, 0.4966, 0.5545, 0.397 , 0.4877,
       0.4951, 0.4877, 0.4958, 0.4045, 0.3301, 0.4951, 0.4508, 0.5173,
       0.4027, 0.4003, 0.4334, 0.2898, 0.4877, 0.4053, 0.5321, 0.4566,
       0.2933, 0.4422, 0.4951, 0.4872, 0.5752, 0.4192, 0.2651, 0.3501,
       0.566 , 0.4022, 0.5123, 0.3514, 0.397 , 0.5545, 0.6107, 0.5694,
       0.4431, 0.4416, 0.4958, 0.3501, 0.4779, 0.2761, 0.404 , 0.5321,
       0.3157, 0.4642, 0.4739, 0.4639, 0.4913, 0.4989, 0.2906, 0.4944,
       0.3329, 0.3301, 0.3586, 0.3301, 0.5168, 0.2447, 0.5215, 0.4607,
       0.5483, 0.4962, 0.4739, 0.4725, 0.5117, 0.497 , 0.4671, 0.5047,
       0.5194, 0.5016, 0.4761, 0.4423, 0.4334, 0.409 , 0.2331, 0.4053,
       0.3176, 0.6244, 0.4664, 0.4592, 0.5215, 0.3514, 0.4322, 0.4949,
       0.2898, 0.4872, 0.4122, 0.3904, 0.4248, 0.2933, 0.5404, 0.406 ,
      

In [12]:
canopy_height_meters = load_canopy_height(geometry=tower_points)
canopy_height_meters

array([2.06429023e+01, 2.21400210e+01, 0.00000000e+00, 1.41648272e+01,
       9.91902963e+00, 4.69379211e-01, 0.00000000e+00, 6.83500353e+00,
       1.79422789e+01, 1.57378972e+01, 6.24472535e+00, 2.25277670e+01,
       0.00000000e+00, 1.52666222e+01, 9.92549452e+00, 1.07070745e+01,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 6.99144429e+00,
       1.79398725e+01, 2.22612354e+01, 0.00000000e+00, 1.71406084e+01,
       6.69737638e-02, 1.91192940e+01, 1.10412274e+01, 0.00000000e+00,
       0.00000000e+00, 1.39596213e+01, 1.46374505e+01, 9.72189605e-02,
       0.00000000e+00, 0.00000000e+00, 2.20106092e-02, 0.00000000e+00,
       1.35347420e+01, 1.75170853e+01, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 2.08070524e+01, 1.96582384e+01, 1.67691536e+01,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       1.58615822e+01, 0.00000000e+00, 1.72034385e+01, 0.00000000e+00,
       0.00000000e+00, 2.08270213e+01, 1.32304601e+01, 1.76799707e+01,
      

In [13]:
field_capacity = load_field_capacity(geometry=tower_points)
field_capacity

array([0.24, 0.37, 0.31, 0.3 , 0.36, 0.33, 0.31, 0.3 , 0.24, 0.26, 0.33,
       0.29, 0.49, 0.22, 0.28, 0.24, 0.25, 0.26, 0.23, 0.21, 0.22, 0.32,
       0.33, 0.23, 0.23, 0.24, 0.23, 0.27, 0.28, 0.25, 0.29, 0.38, 0.33,
       0.2 , 0.33, 0.27, 0.22, 0.31, 0.19, 0.32, 0.33, 0.27, 0.23, 0.27,
       0.26, 0.3 , 0.25, 0.34, 0.32, 0.4 , 0.34, 0.32, 0.36, 0.3 , 0.43,
       0.19, 0.27, 0.35, 0.21, 0.31, 0.32, 0.35, 0.34, 0.19, 0.35, 0.43,
       0.24, 0.28, 0.25, 0.29, 0.31, 0.27, 0.25, 0.28, 0.25, 0.28, 0.3 ,
       0.22, 0.32, 0.25, 0.33, 0.33, 0.25, 0.39, 0.46, 0.38, 0.21, 0.2 ,
       0.31, 0.33, 0.23, 0.28, 0.34, 0.38, 0.25, 0.24, 0.27, 0.38, 0.28,
       0.23, 0.31, 0.33, 0.28, 0.34, 0.29, 0.22, 0.3 , 0.3 , 0.23, 0.32,
       0.34, 0.32, 0.34, 0.29, 0.36, 0.26, 0.31, 0.36, 0.29, 0.28, 0.25])

In [14]:
wilting_point = load_wilting_point(geometry=tower_points)
wilting_point

array([0.11, 0.2 , 0.14, 0.13, 0.11, 0.16, 0.14, 0.1 , 0.11, 0.12, 0.16,
       0.1 , 0.2 , 0.1 , 0.11, 0.1 , 0.11, 0.12, 0.07, 0.08, 0.08, 0.11,
       0.16, 0.08, 0.08, 0.08, 0.09, 0.06, 0.12, 0.07, 0.08, 0.28, 0.16,
       0.08, 0.18, 0.13, 0.08, 0.11, 0.15, 0.09, 0.14, 0.1 , 0.06, 0.08,
       0.1 , 0.1 , 0.1 , 0.17, 0.12, 0.17, 0.14, 0.15, 0.15, 0.11, 0.14,
       0.04, 0.08, 0.18, 0.07, 0.15, 0.15, 0.2 , 0.08, 0.13, 0.13, 0.09,
       0.08, 0.13, 0.1 , 0.1 , 0.15, 0.17, 0.11, 0.13, 0.1 , 0.12, 0.13,
       0.12, 0.11, 0.1 , 0.09, 0.15, 0.09, 0.29, 0.21, 0.13, 0.07, 0.09,
       0.13, 0.15, 0.06, 0.09, 0.18, 0.2 , 0.1 , 0.08, 0.14, 0.11, 0.11,
       0.07, 0.11, 0.14, 0.11, 0.12, 0.15, 0.07, 0.15, 0.1 , 0.1 , 0.15,
       0.14, 0.14, 0.23, 0.12, 0.18, 0.1 , 0.12, 0.16, 0.07, 0.13, 0.12])

In [15]:
tower_static_data_gdf = gpd.GeoDataFrame({
    "ID": tower_IDs,
    "name": tower_names,
    "Topt": Topt,
    "fAPARmax": fAPARmax,
    "canopy_height_meters": canopy_height_meters,
    "field_capacity": field_capacity,
    "wilting_point": wilting_point,
    "geometry": tower_points
})

tower_static_data_gdf

,ID,name,Topt,fAPARmax,canopy_height_meters,field_capacity,wilting_point,geometry
0,US-NC3,NC_Clearcut#3,10.09,0.4659,20.642902,0.24,0.11,POINT (-76.656 35.799)
1,PE-QFR,Quistococha Forest Reserve,25.06,0.5924,22.140021,0.37,0.20,POINT (-73.319 -3.8344)
2,US-Mi3,LTAR UCB (Upper Chesapeake Bay) Miscanthus 3,1.39,0.4865,0.000000,0.31,0.14,POINT (-80.637 41.8222)
3,US-NC4,NC_AlligatorRiver,9.74,0.5777,14.164827,0.30,0.13,POINT (-75.9038 35.7879)
4,CA-DB2,Delta Burns Bog 2,3.43,0.4949,9.919030,0.36,0.11,POINT (-122.9951 49.119)
...,...,...,...,...,...,...,...,...
116,US-xSL,"NEON North Sterling, CO (STER)",0.66,0.3046,0.000000,0.31,0.12,POINT (-103.0293 40.4619)
117,US-xWD,NEON Woodworth (WOOD),0.00,0.3626,0.000000,0.36,0.16,POINT (-99.2414 47.1282)
118,US-CS4,Central Sands Irrigated Agricultural Field,0.00,0.3903,0.000000,0.29,0.07,POINT (-89.5475 44.1597)
119,US-xAE,NEON Klemme Range Research Station (OAES),7.99,0.3387,0.000000,0.28,0.13,POINT (-99.0588 35.4106)


In [16]:
for key in sorted(tower_data_df.keys()):
    print(key)

AirTempC
BESSinst
Date-Time
ESI
ESIrn_BESS
ESIrn_LEcorr50
ESIrn_MOD16
ESIrn_PTJPLSM
ESIrn_STIC
ESIrn_Unc_ECO
ETinst
ETinstUncertainty
EmisWB
G_filt
H_filt
Hcorr25
Hcorr50
Hcorr75
Hcorr_ann
ID
JET
LE
LE_filt
LEcorr25
LEcorr50
LEcorr75
LEcorr_ann
LST
LST_err
MOD16inst
NDVI
NDVI-UQ
NETRAD_filt
PET
PTJPLSMinst
RH
RH_percentage
Rg
Rn
SM
SM_rz
SM_surf
STICinst
SW_IN
Site Name
Ta
Unnamed: 0
albedo
albedo-UQ
climate
eco_time_utc
local_time
solar_hour
solar_time
time_utc
vegetation
view_zenith


In [17]:
observation_tower_IDs = list(tower_data_df.ID)
print(len(observation_tower_IDs))
observation_tower_IDs[:10]

1065


['US-NC3',
 'US-Mi3',
 'US-Mi3',
 'US-Mi3',
 'US-Mi3',
 'US-Mi3',
 'US-NC4',
 'US-Mi1',
 'US-Mi1',
 'US-Mi1']

In [18]:
model_inputs_df = pd.DataFrame({
    "ID": observation_tower_IDs,
})

model_inputs_df = pd.merge(
    left=model_inputs_df, 
    right=tower_static_data_gdf, 
    left_on="ID", 
    right_on="ID", 
    how="left"
)

model_inputs_df

,ID,name,Topt,fAPARmax,canopy_height_meters,field_capacity,wilting_point,geometry
0,US-NC3,NC_Clearcut#3,10.09,0.4659,20.642902,0.24,0.11,POINT (-76.656 35.799)
1,US-Mi3,LTAR UCB (Upper Chesapeake Bay) Miscanthus 3,1.39,0.4865,0.000000,0.31,0.14,POINT (-80.637 41.8222)
2,US-Mi3,LTAR UCB (Upper Chesapeake Bay) Miscanthus 3,1.39,0.4865,0.000000,0.31,0.14,POINT (-80.637 41.8222)
3,US-Mi3,LTAR UCB (Upper Chesapeake Bay) Miscanthus 3,1.39,0.4865,0.000000,0.31,0.14,POINT (-80.637 41.8222)
4,US-Mi3,LTAR UCB (Upper Chesapeake Bay) Miscanthus 3,1.39,0.4865,0.000000,0.31,0.14,POINT (-80.637 41.8222)
...,...,...,...,...,...,...,...,...
1060,US-xAE,NEON Klemme Range Research Station (OAES),7.99,0.3387,0.000000,0.28,0.13,POINT (-99.0588 35.4106)
1061,US-xAE,NEON Klemme Range Research Station (OAES),7.99,0.3387,0.000000,0.28,0.13,POINT (-99.0588 35.4106)
1062,US-xAE,NEON Klemme Range Research Station (OAES),7.99,0.3387,0.000000,0.28,0.13,POINT (-99.0588 35.4106)
1063,US-xAE,NEON Klemme Range Research Station (OAES),7.99,0.3387,0.000000,0.28,0.13,POINT (-99.0588 35.4106)


In [19]:
ST_K = np.array(tower_data_df.LST)
model_inputs_df["ST_K"] = ST_K
print(np.nanmin(ST_K), np.nanmean(ST_K), np.nanmax(ST_K))

258.72 302.1800469295775 359.26


In [20]:
ST_C = ST_K - 273.15
model_inputs_df["ST_C"] = ST_C
print(np.nanmin(ST_C), np.nanmean(ST_C), np.nanmax(ST_C))

-14.42999999999995 29.030046929577487 86.11000000000001


In [21]:
NDVI = np.array(tower_data_df.NDVI)
model_inputs_df["NDVI"] = NDVI
print(np.nanmin(NDVI), np.nanmean(NDVI), np.nanmax(NDVI))

-0.02429185 0.4528892517239436 0.94546


In [22]:
Ta_C = np.array(tower_data_df.Ta)
model_inputs_df["Ta_C"] = Ta_C
print(np.nanmin(Ta_C), np.nanmean(Ta_C), np.nanmax(Ta_C))

-14.605048 22.321587967441317 39.710495


In [23]:
RH = np.array(tower_data_df.RH)
model_inputs_df["RH"] = RH
print(np.nanmin(RH), np.nanmean(RH), np.nanmax(RH))

0.27253073 0.42692000418779347 0.98366296


In [24]:
Rg = np.array(tower_data_df.Rg)
model_inputs_df["Rg"] = Rg
print(np.nanmin(Rg), np.nanmean(Rg), np.nanmax(Rg))

-23.763361 606.9121518676056 1042.9371


In [25]:
Rn = np.array(tower_data_df.Rn)
model_inputs_df["Rn"] = Rn
print(np.nanmin(Rn), np.nanmean(Rn), np.nanmax(Rn))

0.0 414.2791522507042 763.1359


In [26]:
albedo = np.array(tower_data_df.albedo)
model_inputs_df["albedo"] = albedo
print(np.nanmin(albedo), np.nanmean(albedo), np.nanmax(albedo))

0.015407953 0.10903688726384977 0.6173986


In [27]:
SM = np.array(tower_data_df.SM)
model_inputs_df["SM"] = SM
print(np.nanmin(SM), np.nanmean(SM), np.nanmax(SM))

0.0 0.16790158060469482 0.89711


In [28]:
EmisWB = np.array(tower_data_df.EmisWB)
model_inputs_df["EmisWB"] = EmisWB
print(np.nanmin(EmisWB), np.nanmean(EmisWB), np.nanmean(EmisWB))

0.734 0.9682704225821597 0.9682704225821597


In [29]:
model_inputs_df

,ID,name,Topt,fAPARmax,canopy_height_meters,field_capacity,wilting_point,geometry,ST_K,ST_C,NDVI,Ta_C,RH,Rg,Rn,albedo,SM,EmisWB
0,US-NC3,NC_Clearcut#3,10.09,0.4659,20.642902,0.24,0.11,POINT (-76.656 35.799),305.10,31.95,0.709729,32.658920,0.560215,545.51056,393.85710,0.215445,0.192359,0.948
1,US-Mi3,LTAR UCB (Upper Chesapeake Bay) Miscanthus 3,1.39,0.4865,0.000000,0.31,0.14,POINT (-80.637 41.8222),304.34,31.19,0.605842,24.227982,0.458503,848.34390,643.47500,0.117238,0.293216,0.952
2,US-Mi3,LTAR UCB (Upper Chesapeake Bay) Miscanthus 3,1.39,0.4865,0.000000,0.31,0.14,POINT (-80.637 41.8222),304.06,30.91,0.578494,26.178862,0.621679,838.81160,628.76870,0.117280,0.433436,0.972
3,US-Mi3,LTAR UCB (Upper Chesapeake Bay) Miscanthus 3,1.39,0.4865,0.000000,0.31,0.14,POINT (-80.637 41.8222),301.80,28.65,0.658499,22.527096,0.623986,851.72480,626.95960,0.084629,0.391243,0.974
4,US-Mi3,LTAR UCB (Upper Chesapeake Bay) Miscanthus 3,1.39,0.4865,0.000000,0.31,0.14,POINT (-80.637 41.8222),303.18,30.03,0.666747,23.280691,0.571613,702.55160,513.13380,0.120526,0.333870,0.960
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1060,US-xAE,NEON Klemme Range Research Station (OAES),7.99,0.3387,0.000000,0.28,0.13,POINT (-99.0588 35.4106),278.78,5.63,0.268441,3.815752,0.395740,286.84660,163.24739,0.092853,0.085971,0.980
1061,US-xAE,NEON Klemme Range Research Station (OAES),7.99,0.3387,0.000000,0.28,0.13,POINT (-99.0588 35.4106),293.28,20.13,0.225175,19.266186,0.397430,290.87400,166.19196,0.111844,0.086697,0.976
1062,US-xAE,NEON Klemme Range Research Station (OAES),7.99,0.3387,0.000000,0.28,0.13,POINT (-99.0588 35.4106),301.94,28.79,0.282996,32.110336,0.341227,352.19530,235.88159,0.106782,0.058330,0.976
1063,US-xAE,NEON Klemme Range Research Station (OAES),7.99,0.3387,0.000000,0.28,0.13,POINT (-99.0588 35.4106),290.72,17.57,0.284051,10.464681,0.349086,420.67880,247.87890,0.106775,0.067998,0.976


In [30]:
model_inputs_df.keys()

Index(['ID', 'name', 'Topt', 'fAPARmax', 'canopy_height_meters',
       'field_capacity', 'wilting_point', 'geometry', 'ST_K', 'ST_C', 'NDVI',
       'Ta_C', 'RH', 'Rg', 'Rn', 'albedo', 'SM', 'EmisWB'],
      dtype='object')

In [31]:
model_inputs_df.albedo

0       0.215445
1       0.117238
2       0.117280
3       0.084629
4       0.120526
          ...   
1060    0.092853
1061    0.111844
1062    0.106782
1063    0.106775
1064    0.113165
Name: albedo, Length: 1065, dtype: float64

In [32]:
results = process_PTJPLSM_table(model_inputs_df)
results

[2025-07-21 17:48:35 INFO] variable fAPARmax min: 0.233 mean: 0.382 max: 0.624 nan: 0.00% (nan)
[2025-07-21 17:48:35 INFO] GEOS-5 FP working directory: /Users/gregoryhalverson/data/GEOS5FP
[2025-07-21 17:48:35 INFO] GEOS-5 FP download directory: ~/data/GEOS5FP
[2025-07-21 17:48:35 INFO] variable Ta_C min: -14.605 mean: 22.322 max: 39.710 nan: 0.00% (nan)
[2025-07-21 17:48:35 INFO] variable RH min: 0.273 mean: 0.427 max: 0.984 nan: 0.00% (nan)
[2025-07-21 17:48:35 INFO] variable soil_moisture min: 0.000 mean: 0.168 max: 0.897 nan: 0.00% (nan)
[2025-07-21 17:48:35 INFO] variable field_capacity min: 0.190 mean: 0.283 max: 0.390 nan: 0.00% (nan)
[2025-07-21 17:48:35 INFO] variable wilting_point min: 0.040 mean: 0.121 max: 0.290 nan: 0.00% (nan)
[2025-07-21 17:48:35 INFO] variable canopy_height_meters min: 0.000 mean: 6.044 max: 30.192 nan: 0.00% (nan)
[2025-07-21 17:48:35 INFO] variable Rn_Wm2 min: 0.000 mean: 414.279 max: 763.136 nan: 0.00% (nan)
[2025-07-21 17:48:35 INFO] variable G min:

/Users/gregoryhalverson/Projects/PT-JPL-SM/PTJPLSM/partitioning/fREW.py:36: RuntimeWarning: divide by zero encountered in divide
  fREW = rt.clip(rt.where(FCWP == 0, 0, SMWP / FCWP), 0, 1)
/Users/gregoryhalverson/Projects/PT-JPL-SM/PTJPLSM/partitioning/fREW.py:36: RuntimeWarning: invalid value encountered in divide
  fREW = rt.clip(rt.where(FCWP == 0, 0, SMWP / FCWP), 0, 1)
/Users/gregoryhalverson/Projects/PT-JPL-SM/PTJPLSM/partitioning/fTRM.py:28: RuntimeWarning: divide by zero encountered in divide
  WPCH = rt.clip(rt.where(CHscalar == 0, 0, wilting_point / CHscalar), 0, 1)
/Users/gregoryhalverson/Projects/PT-JPL-SM/PTJPLSM/partitioning/fTRM.py:30: RuntimeWarning: invalid value encountered in power
  fTREW = rt.clip(1 - ((CR - soil_moisture) / (CR - WPCH)) ** CHscalar, 0, 1)


,ID,name,Topt,fAPARmax,canopy_height_meters,field_capacity,wilting_point,geometry,ST_K,ST_C,...,SM,EmisWB,G,Rn_soil,LE_soil,Rn_canopy,PET,LE_canopy,LE_interception,LE
0,US-NC3,NC_Clearcut#3,10.09,0.4659,20.642902,0.24,0.11,POINT (-76.656 35.799),305.10,31.95,...,0.192359,0.948,51.001527,108.025913,57.976355,285.831187,348.579227,225.176910,28.623222,311.776487
1,US-Mi3,LTAR UCB (Upper Chesapeake Bay) Miscanthus 3,1.39,0.4865,0.000000,0.31,0.14,POINT (-80.637 41.8222),304.34,31.19,...,0.293216,0.952,81.309890,242.983670,149.111718,400.491330,518.484847,222.917805,16.324344,388.353867
2,US-Mi3,LTAR UCB (Upper Chesapeake Bay) Miscanthus 3,1.39,0.4865,0.000000,0.31,0.14,POINT (-80.637 41.8222),304.06,30.91,...,0.433436,0.972,80.764205,255.079320,165.019858,373.689380,518.782458,92.778192,52.841827,310.639877
3,US-Mi3,LTAR UCB (Upper Chesapeake Bay) Miscanthus 3,1.39,0.4865,0.000000,0.31,0.14,POINT (-80.637 41.8222),301.80,28.65,...,0.391243,0.974,64.855799,203.478663,124.726106,423.480937,505.753641,99.834306,57.763947,282.324359
4,US-Mi3,LTAR UCB (Upper Chesapeake Bay) Miscanthus 3,1.39,0.4865,0.000000,0.31,0.14,POINT (-80.637 41.8222),303.18,30.03,...,0.333870,0.960,58.296844,162.335322,94.663736,350.798478,413.852320,118.000423,34.076662,246.740821
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1060,US-xAE,NEON Klemme Range Research Station (OAES),7.99,0.3387,0.000000,0.28,0.13,POINT (-99.0588 35.4106),278.78,5.63,...,0.085971,0.980,4.103044,121.450797,1.670860,41.796593,92.388700,13.741885,0.595122,16.007867
1061,US-xAE,NEON Klemme Range Research Station (OAES),7.99,0.3387,0.000000,0.28,0.13,POINT (-99.0588 35.4106),293.28,20.13,...,0.086697,0.976,15.442521,131.899739,2.478684,34.292221,128.608037,19.762245,0.729878,22.970807
1062,US-xAE,NEON Klemme Range Research Station (OAES),7.99,0.3387,0.000000,0.28,0.13,POINT (-99.0588 35.4106),301.94,28.79,...,0.058330,0.976,30.976170,171.573784,1.927835,64.307806,207.238994,53.861595,0.881771,56.671201
1063,US-xAE,NEON Klemme Range Research Station (OAES),7.99,0.3387,0.000000,0.28,0.13,POINT (-99.0588 35.4106),290.72,17.57,...,0.067998,0.976,19.863577,180.002902,1.679801,67.875998,161.062364,39.378499,0.711993,41.770294


In [33]:
model_inputs_df.to_csv(generated_input_table_filename, index=False)